<div class='alert alert-info' style='text-align: center'><h1>Drawing Reference Lines On MR Images</h1>
    - yet another MR notebook -
    </div>
    
#### The goal of this notebook is to demonstrate how images within a study are spatially related to one another.

- We'll use the ImageOrientationPatient and ImagePositionPatient tags to calculate and draw a reference line.
- This is a simple demo of the technique. There are some scaling and spatial orientation steps that should be added for completeness, but this will do in the context of this comp.

In [ ]:
import os
import numpy as np
import pydicom
import matplotlib.pyplot as plt
import cv2

#### Define a function that determines the reference line position from the position and orientation tags of a 'source' and 'destination' image.

- Since we're projecting a 2D image into 3D space, we'll actually make a box that represents all four sides of the source image.
- If the two images are orthogonal to each other in at least two planes, the sides of the 2D image will overlap and we'll see a single line drawn on the destination image.
- It's important to understand the difference in the patient's coordinates (3D) and the image's coordinates (2D). 
- Here is a detailed explanation of how the ImagePositionPatient and ImageOrientationPatient tags work to orient the image. -> http://dicomiseasy.blogspot.com/2013/06/getting-oriented-using-image-plane.html

In [ ]:
# Get the relavent tags and calculate the coordinates that specify the 'box' that one image projects onto another
def create_reference_line(src, dst):
    dst_iop = dst.ImageOrientationPatient
    dst_ipp = dst.ImagePositionPatient
    src_iop = src.ImageOrientationPatient
    src_ipp = src.ImagePositionPatient  

    pos_x = []
    pos_y = []
    pos_z = []
    row_pixel = []
    col_pixel = []

    row_len = int(dst.Rows * dst.PixelSpacing[1])
    col_len = int(dst.Columns * dst.PixelSpacing[0])
    
    # Get the coordinates of the box
    pos_x.append(src_ipp[0]) # Top Left Corner
    pos_y.append(src_ipp[1])
    pos_z.append(src_ipp[2])
    pos_x.append(src_ipp[0] + src_iop[0] * row_len) # Top Right Corner
    pos_y.append(src_ipp[1] + src_iop[1] * row_len)
    pos_z.append(src_ipp[2] + src_iop[2] * row_len)
    pos_x.append(src_ipp[0] + src_iop[0] * row_len + src_iop[3] * col_len) # Bottom Right Corner
    pos_y.append(src_ipp[1] + src_iop[1] * row_len + src_iop[4] * col_len)
    pos_z.append(src_ipp[2] + src_iop[2] * row_len + src_iop[5] * col_len)
    pos_x.append(src_ipp[0] + src_iop[3] * col_len) # Bottom Left Corner
    pos_y.append(src_ipp[1] + src_iop[4] * col_len)
    pos_z.append(src_ipp[2] + src_iop[5] * col_len)
    
    for i in range (0,3):
    
        pos_x[i] -= dst_ipp[0]
        pos_y[i] -= dst_ipp[1]
        pos_z[i] -= dst_ipp[2]
        
        # Rotate the coordinates in 3D space
        cp = int(dst_iop[0] * pos_x[i] + dst_iop[1] * pos_y[i] + dst_iop[2] * pos_z[i])
        rp = int(dst_iop[3] * pos_x[i] + dst_iop[4] * pos_y[i] + dst_iop[5] * pos_z[i])

        col_pixel.append(int(cp / float(dst.PixelSpacing[0])))
        row_pixel.append(int(rp / float(dst.PixelSpacing[1])))
    
    return col_pixel, row_pixel

#### Now we'll read in a couple images from the same study, but different series. 
- We want series in different planes, as it doesn't make sense to display reference lines on coplanar images.
- Technically, we should verify the FrameOfReferenceUID tags match, which don't exist in this datatset, so we'll pretend they do.

In [ ]:
# Read in two images from the same study, different series.
src_img = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00006/T1w/Image-15.dcm')
dst_img = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00006/T2w/Image-200.dcm')

# Print these two tags so we can get used to seeing them. Nerds will be able to determine orientation by just looking at the IOP tags (after some time).
print("Src IOP: ", src_img.ImageOrientationPatient)
print("Dest IOP: ", dst_img.ImageOrientationPatient)

# Crunch the destination image's pixels down to 8 bit so we can draw an 8 bit line on it
pixels = dst_img.pixel_array
pixels = pixels - np.min(pixels)
pixels = pixels / np.max(pixels)
pixels = (pixels * 255).astype(np.uint8)

# Get the line (box) coords
col_pixel, row_pixel = create_reference_line(src_img, dst_img)

# Draw lines on the destination image
for i in range(0,2):
    dst_img = cv2.line(pixels, (col_pixel[i], row_pixel[i]), (col_pixel[i+1], row_pixel[i+1]), (255, 255, 255), 2)
    
# Plot the results
fig, axes = plt.subplots(nrows=1, ncols=2,sharex=True, sharey=True, figsize=(14, 7))
ax = axes.ravel()

ax[0].set_title('Source Image (T1w - Axial)')
ax[0].imshow(src_img.pixel_array, cmap='gray');
ax[1].set_title('Destination Image (T2w - Sagittal)')
ax[1].imshow(dst_img, cmap='gray');

#### We can see from the printout above the images that the source image IOP cosines are all either 0 or 1. The destination cosines are not whole numbers.
- This means the source image **is** at right angles relative to the patient's body.
- The destination image **is not** positioned at right angles to the patient's body.

#### Now, we'll plot the same two images, but reverse their order so we can see this.

In [ ]:
# Read in two images from the same study (technically, we should verify the StudyInstanceUID and SeriesInstanceUIDs match, but we'll assume they do here)
src_img = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00006/T2w/Image-200.dcm')
dst_img = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00006/T1w/Image-15.dcm')

print("Src IOP: ", src_img.ImageOrientationPatient)
print("Dest IOP: ", dst_img.ImagePositionPatient)

# Crunch the destination image's pixels down to 8 bit so we can draw an 8 bit line on it
pixels = dst_img.pixel_array
pixels = pixels - np.min(pixels)
pixels = pixels / np.max(pixels)
pixels = (pixels * 255).astype(np.uint8)

# Get the line (box) coords
col_pixel, row_pixel = create_reference_line(src_img, dst_img)

# Draw lines on the destination image
for i in range(0,2):
    dst_img = cv2.line(pixels, (col_pixel[i], row_pixel[i]), (col_pixel[i+1], row_pixel[i+1]), (255, 255, 255), 2)
    
# Plot the results
fig, axes = plt.subplots(nrows=1, ncols=2,sharex=True, sharey=True, figsize=(14, 7))
ax = axes.ravel()

ax[0].set_title('Source Image (T2w - Sagittal)')
ax[0].imshow(src_img.pixel_array, cmap='gray');
ax[1].set_title('Destination Image (T1w - Axial)')
ax[1].imshow(dst_img, cmap='gray');

#### We can see when we project the T2w sagittal series onto the T1w axial series, it's tilted slightly.
- This is because the technologist adjusted the reconstruction of this series to fit the patient who wasn't laying perfectly straight inside the magnet.

#### Let's add another series from this study

In [ ]:
# Read in two images 
src_img = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00006/FLAIR/Image-455.dcm')
dst_img = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00006/T2w/Image-200.dcm')

print("Src IOP: ", src_img.ImageOrientationPatient)
print("Dest IOP: ", dst_img.ImagePositionPatient)

# Crunch the destination image's pixels down to 8 bit so we can draw an 8 bit line on it
pixels = dst_img.pixel_array
pixels = pixels - np.min(pixels)
pixels = pixels / np.max(pixels)
pixels = (pixels * 255).astype(np.uint8)

# Get the line (box) coords
col_pixel, row_pixel = create_reference_line(src_img, dst_img)

# Draw lines on the destination image
for i in range(0,2):
    dst_img = cv2.line(pixels, (col_pixel[i], row_pixel[i]), (col_pixel[i+1], row_pixel[i+1]), (255, 255, 255), 2)
    
# Plot the results
fig, axes = plt.subplots(nrows=1, ncols=2,sharex=True, sharey=True, figsize=(14, 7))
ax = axes.ravel()

ax[0].set_title('Source Image (FLAIR - Axial)')
ax[0].imshow(src_img.pixel_array, cmap='gray');
ax[1].set_title('Destination Image (T2w - Sagittal)')
ax[1].imshow(dst_img, cmap='gray');

#### This one is slightly tilted as well. 
- It's difficult to tell by the line, but you can see from the IOP numbers, that neither of these series are orthogonal to the patient.

#### Let's go back to the T2w (sagittal) and T1w (coronal) combo and plot a bunch of reference lines.

In [ ]:
# Load the destination image
dst = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00006/T1w/Image-11.dcm')
pixels = dst.pixel_array
pixels = pixels - np.min(pixels)
pixels = pixels / np.max(pixels)
pixels = (pixels * 255).astype(np.uint8)

# Load a bunch of images from the source series and plot their lines onto the dest image
for i in range(100, 300, 10):
    src = pydicom.dcmread('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00006/T2w/Image-' + str(i) + '.dcm')

    # Get the line (box) coords
    col_pixel, row_pixel = create_reference_line(src, dst)

    # Draw lines on the destination image
    for i in range(0,2):
        pixels = cv2.line(pixels, (col_pixel[i], row_pixel[i]), (col_pixel[i+1], row_pixel[i+1]), (255, 255, 255), 2)
    
# Plot the results
fig, axes = plt.subplots(figsize=(14, 7))
plt.imshow(pixels, cmap='gray');

#### This shows the orientation of every tenth image of the T2w sagittals on the T1w axial, starting at image 100 and ending at image 300.
- You could loop through the entire series, remove empty images etc.

#### Some of my other MR notebooks
- Tumor Object Detection -> https://www.kaggle.com/davidbroberts/brain-tumor-object-detection
- Determining MR Slice Orientation -> https://www.kaggle.com/davidbroberts/determining-mr-slice-orientation
- Determining MR image planes -> https://www.kaggle.com/davidbroberts/determining-mr-image-planes
- Determining DICOM image order -> https://www.kaggle.com/davidbroberts/determining-dicom-image-order
- Manual VOI LUT on MR images -> https://www.kaggle.com/davidbroberts/manual-voi-lut-on-mr-images
- Standardizing MR Images -> https://www.kaggle.com/davidbroberts/standardizing-mr-images
- Export DICOM Images by Plane -> https://www.kaggle.com/davidbroberts/export-dicom-series-by-plane